# Mnemonics & HD Wallets - Examples with Libbitcoin BX
In this BX tutorial, we create mnemonic backup phrases, from which HD private and public keys can be derived for use. The mnemonic phrase can later be used to recover hd keys used in a previous wallet with the same seed.

## 1) Seeding an HD wallet.
<img src="images/hd_mnemonic_to_master.jpg" alt="drawing" style="" width="700px"/>

### 1.1) Deriving the mnemonic phrase (BIP39)

In [ ]:
# 128, 160, 192, 224, 256 bits of entropy.
bx seed --bit_length 128


In [ ]:
bx mnemonic-new --language ja fc3044d027dbae56c1b156ba3158f55c60f982370508909622806b680e86b787


In [ ]:
# mnemonic-to-seed always derives a 512-bit long seed.
bx mnemonic-to-seed --language ja むける りりく あんぜん ひろい よかぜ いっぽう でぬかえ むいか うんてん げいのうじん ひほう きぞく


## 1.2) Deriving the master HD keys (BIP32)

In [ ]:
# accepts seed 128bit and longer.
bx hd-new b7188b58f581621725b1685e6a8fb5c536469b0893c52846753df856ef561ed18173f4d4e4f7aab96cd8b588b0894eaea702db7fff19e2ac06ec78ad1aaf8443


In [ ]:
bx hd-to-public tprv8ZgxMBicQKsPdSeZJzyvz6sEVDV8iMJfuKczswWUFTPXa2eTVTg96qh1VVsPC1fDR37gh6KUjo9V4i3peYUPuyP7t5esiH7Lnja8CUNEMFS


## 2) Deriving HD children
<img src="images/hd_children_derivation.jpg" alt="drawing" style="" width="700px"/>

### 2.1) Deriving unhardened children hd keys.

**`m` &#8658; `m/0/1/2`** `(private-key child derivation)`

In [ ]:
bx hd-private --index 0 tprv8ZgxMBicQKsPdSeZJzyvz6sEVDV8iMJfuKczswWUFTPXa2eTVTg96qh1VVsPC1fDR37gh6KUjo9V4i3peYUPuyP7t5esiH7Lnja8CUNEMFS \
| bx hd-private --index 1 \
| bx hd-private --index 2 


**`M` &#8658; `M/0/1/2`** `(public-key child derivation)`

In [ ]:
bx hd-public --index 0  tpubD6NzVbkrYhZ4WugMCeeXPWXM4F14sgVaUdDnATYmfjBvQWuE7rVjHLJsfdv4zdAnbMRqy9JqZZ6AatQiDNETt1kgRQj4ch1nGLsqX2qdyEK \
| bx hd-public --index 1 \
| bx hd-public --index 2 


**`m/0/1/2` &#8658; `M/0/1/2`** 

In [ ]:
#(compare key with preceding example)
bx hd-to-public tprv8hCwKRwDxELVyNjavsc5vq5au9HbBE8XgMPRMMS1qJwVru3B7DpTqjXEU28Ctrw8UYxQXeEvMHM6vPTGGgcTWjMTzgdyb8kHouM7UPhdKiS


**Payment addresses from hd-keys**
* `hd-to-ec` derives public key from extended public key.
* `ec-to-address` returns payment addresss from public key.
    * `--version` : `0x6f` = `111` in decimal

In [ ]:
bx hd-to-ec tpubDDtyTqyU6c2ArqmNpXGgLEjhUAoXLZKSFezCdsUKFajthPHwjce42E96eAQzmny1cXPw8Ndpe6mehZez5AYSXm2RWVi6p8eabeFMDrDAMqc \
| bx ec-to-address --version 111


### 2.2) Deriving hardened children hd keys.

**`m` &#8658; `m/44'`** 

In [ ]:
bx hd-private --hard --index 44 tprv8ZgxMBicQKsPdSeZJzyvz6sEVDV8iMJfuKczswWUFTPXa2eTVTg96qh1VVsPC1fDR37gh6KUjo9V4i3peYUPuyP7t5esiH7Lnja8CUNEMFS


**`m` &#8658; `m/44'` &#8658; `M/44'`** 

In [ ]:
bx hd-public --hard --index 44 tprv8ZgxMBicQKsPdSeZJzyvz6sEVDV8iMJfuKczswWUFTPXa2eTVTg96qh1VVsPC1fDR37gh6KUjo9V4i3peYUPuyP7t5esiH7Lnja8CUNEMFS


**`m/44'` &#8658; `m/44'/1'` &#8658; `M/44'/1'`** 

In [ ]:
bx hd-public --hard --index 1 tprv8cJ2rro1fDs44qYPwWi5eMHgDZ2fdEGRtXYUsXzBJP53p3hPN3b1mQcigUU3H85tA1zFYsRvXVRumhhDDo7y2v8RS22EvrbyoQuFHmWE4CZ


**Try: `M/44'` &#8658; `M/44'/1'`**

In [ ]:
bx hd-public --hard --index 1 tpubD9pU5CBaLtLYDonezhUaV1MfbYKytU1UsdkWpkFjEkFTBGCwX9Ma5nrufxte6NR6K78XpPWoTXSzhEFrRnGDyM8FMFVbzc8jqcMAim2aBZd 


## 3) Recovering a HD wallet from mnemonic phrase
<img src="images/hd_wallet_recovery.jpg" alt="drawing" style="" width="700px"/>

**In order to recover previously used hd-derivation paths of an hd-wallet:**
* Derive hd parent keys of receiving & change addresses from mnemonic (& passphrase).
    * Parent of receiving addresses
        * `m/44'/0'/0'/0` (mainnet)
        * `m/44'/1'/0'/0` (testnet)
    * Parent of change addresses
        * `m/44'/0'/0'/1` (mainnet)
        * `m/44'/1'/0'/1` (testnet)        
* Increment hd-key index of children until unused addresses are found.
    * Query Libbitcoin Server to check history of a payment address
        * `bx fetch-history` `[payment address]`

In [ ]:
mnemonic='むける りりく あんぜん ひろい よかぜ いっぽう でぬかえ むいか うんてん げいのうじん ひほう きぞく'
m_testnet=$(bx mnemonic-to-seed --language ja $mnemonic | bx hd-new)
m_44h_1h_0h_0=$(bx hd-private --index 44 --hard $m_testnet \
| bx hd-private --index 1 --hard \
| bx hd-private --index 0 --hard \
| bx hd-private --index 0)


gap_count=0
current_index=0

while (( $gap_count < 20 ))
do

    current_address=$(bx hd-private --index $current_index $m_44h_1h_0h_0 \
    | bx hd-to-public \
    | bx hd-to-ec \
    | bx ec-to-address --version 111)
     
    query_address_history_reply=$(bx fetch-history $current_address)
    
    if  [ "$query_address_history_reply" = 'transfers ""' ]
    then
        echo address at index m_44h_1h_0h_0_$current_index is unused: $current_address 
        let gap_count++
    
    # reset gap counter if gap interrupted before it reaches 20.
    elif (( $gap_count > 0 ))
    then 
        gap_count=0
        echo address at index m_44h_1h_0h_0_$current_index is used: $current_address 
    else
        echo address at index m_44h_1h_0h_0_$current_index is used: $current_address 
    fi
    
    let current_index++
    
done